# Cleaning_any_asylum.ipynb
#### This script cleans and merges relevant variables across datasets
#### considers a grant on any asylum case type (full, witholding, wcat) a "grant" decision on the case. 
#### Currently, it is doing cleaning and merging only for the baseline model.

In [1]:
import pandas as pd
import numpy as np
pd.set_option('precision', 5)

In [2]:
path = '/data/Dropbox/Data/Asylum_Courts/raw'

## Clean court_appln.csv

relevant variables: idnProceeding, idnCase, Appl_Code

In [3]:
app = pd.read_csv(path + '/court_appln.csv', low_memory=False)

# descriptive stats
#app.count()
#app.describe()

In [4]:
# adding placeholder number to count how many applications were filed for idnProceeding
app['numAppsPerProc'] = 1
app['numAppsPerProc'] = app['numAppsPerProc'].astype('int64')

# adding additional feature based on how many applications have been filed for the same (idnCase, idnProceeding) pair
app['numAppsPerProc'] = app.groupby(['idnCase', 'idnProceeding'])['numAppsPerProc'].transform('count')

# dropping all applications with empty decisions
app = app.dropna(subset=['Appl_Dec'])

#app.describe()
app.count()

idnProceedingAppln    4232861
idnProceeding         4232861
idnCase               4232857
Appl_Code             4232859
Appl_Recd_Date        4232826
Appl_Dec              4232861
numAppsPerProc        4232857
dtype: int64

In [5]:
# making a new variable, dec, simplifying grant decisions to DENY, GRANT, or nan
app['dec']= np.nan
app.loc[app.Appl_Dec.isin(['G','F','N','L','C']),'dec']= 'GRANT'
app.loc[(app["Appl_Dec"] == 'D'),'dec'] = 'DENY'
app = app[app.dec.isin(['DENY','GRANT'])] # only include DENY or GRANT cases

app.count()


idnProceedingAppln    2374977
idnProceeding         2374977
idnCase               2374977
Appl_Code             2374976
Appl_Recd_Date        2374951
Appl_Dec              2374977
numAppsPerProc        2374977
dec                   2374977
dtype: int64

In [6]:
# only keep applications of type ASYL, ASYW, WCAT. sort by Grant, then deny, then case type in order (ASYL, ASYW, WCAT)
#, then date within idnproceeding
#sorting by date--if there are multiple applications with the same decision with the same case type, 
#take the most recent one.
app = app[app.Appl_Code.isin(['ASYL','ASYW', 'WCAT'])]

#sort multiple times because some need to be ascending and some descending
app = app.sort_values(['idnProceeding','dec','Appl_Code','Appl_Recd_Date'],ascending=[True,False,True,False])


In [7]:
app = app.rename(columns={"idnCase":"idncase", "idnProceeding":"idnproceeding"})

In [8]:
app.head(10)

,idnProceedingAppln,idnproceeding,idncase,Appl_Code,Appl_Recd_Date,Appl_Dec,numAppsPerProc,dec
40,41,75.0,3328085.0,ASYL,1994-12-20 00:00:00,D,3.0,DENY
836171,837950,75.0,3328085.0,ASYW,1994-12-20 00:00:00,D,3.0,DENY
42,43,85.0,3328111.0,ASYL,1995-04-02 00:00:00,D,3.0,DENY
836173,837952,85.0,3328111.0,ASYW,1995-04-02 00:00:00,D,3.0,DENY
48,49,103.0,3328153.0,ASYL,1995-05-05 00:00:00,G,2.0,GRANT
51,52,111.0,3328175.0,ASYL,1995-03-31 00:00:00,D,3.0,DENY
55,56,136.0,3327844.0,ASYL,1995-03-06 00:00:00,D,3.0,DENY
836186,837965,136.0,3327844.0,ASYW,1995-03-06 00:00:00,D,3.0,DENY
57,58,139.0,3327852.0,ASYL,1996-05-14 00:00:00,G,2.0,GRANT
836188,837967,139.0,3327852.0,ASYW,1996-05-14 00:00:00,G,2.0,GRANT


In [9]:
# make unique--take the first application for each proceeding, when sorted in order dec (grant deny),
#case type(ASYL, ASYW, WCAT), date
app2 = app.groupby('idnproceeding', as_index=False).first()


In [10]:
app2.count()

idnproceeding         614388
idnProceedingAppln    614388
idncase               614388
Appl_Code             614388
Appl_Recd_Date        614387
Appl_Dec              614388
numAppsPerProc        614388
dec                   614388
dtype: int64

In [11]:
app2.head(10)

,idnproceeding,idnProceedingAppln,idncase,Appl_Code,Appl_Recd_Date,Appl_Dec,numAppsPerProc,dec
0,75.0,41,3328085.0,ASYL,1994-12-20 00:00:00,D,3.0,DENY
1,85.0,43,3328111.0,ASYL,1995-04-02 00:00:00,D,3.0,DENY
2,103.0,49,3328153.0,ASYL,1995-05-05 00:00:00,G,2.0,GRANT
3,111.0,52,3328175.0,ASYL,1995-03-31 00:00:00,D,3.0,DENY
4,136.0,56,3327844.0,ASYL,1995-03-06 00:00:00,D,3.0,DENY
5,139.0,58,3327852.0,ASYL,1996-05-14 00:00:00,G,2.0,GRANT
6,145.0,60,3327869.0,ASYL,1995-11-29 00:00:00,D,3.0,DENY
7,147.0,61,3327877.0,ASYL,1995-11-29 00:00:00,D,3.0,DENY
8,149.0,62,3327884.0,ASYL,1995-04-14 00:00:00,D,3.0,DENY
9,159.0,63,3327927.0,ASYL,1995-07-25 00:00:00,D,3.0,DENY


## Clean master.csv

Relevant variables: idncase, idnproceeding, osc_date, tracid, nat

In [12]:
# load in data
master = pd.read_csv(path + '/master.csv', low_memory=False)
#master.describe() # summary stats

In [13]:
#drop empty cases and proceedings
master = master.dropna(subset= ['idncase','idnproceeding'])
#master.describe()

In [14]:
# stuff on osc_date (date charges filed or NTA)
master = master.dropna(subset=['osc_date']) # dropping empty dates

master['osc_date'] = master['osc_date'].astype('str')
master = master[master['osc_date'].apply(lambda x: len(x) == 9)] # delete dates invalid formats

master['osc_date'] = pd.to_datetime(master['osc_date'], format='%d%b%Y') # change to date format 
#master.describe()

# delete NTA dates before 1984
master = master[master.osc_date.dt.year>1983]

In [15]:
#comp date (date proceeding completed)
master = master.dropna(subset=['comp_date']) # dropping empty dates

master['comp_date'] = master['comp_date'].astype('str')
master = master[master['comp_date'].apply(lambda x: len(x) == 9)] # delete dates invalid formats

master['comp_date'] = pd.to_datetime(master['comp_date'], format='%d%b%Y') # change to date format 

#drop comp date dates before 1985
master = master[master.comp_date.dt.year>1984]


In [16]:
master.head(10)

,idncase,nat,case_type,c_asy_type,idnproceeding,base_city_code,hearing_loc_code,dec_type,dec_code,other_comp,osc_date,input_date,comp_date,attorney_flag,ij_code,tracid
11,2046920.0,MX,RMV,NaN,3200048,CHI,CHD,O,X,NaN,2004-08-06,11AUG2004,2004-08-11,NaN,RDV,31.0
12,2046921.0,MX,RMV,NaN,3200049,CHI,CHD,O,X,NaN,2004-08-06,10AUG2004,2004-08-11,NaN,RDV,31.0
13,2046922.0,MX,RMV,NaN,3200050,CHI,CHD,O,X,NaN,2004-08-09,19AUG2004,2004-08-19,NaN,JLG,29.0
14,2046923.0,PL,RMV,NaN,3200051,CHI,CHD,O,X,NaN,2004-08-09,13AUG2004,2004-08-25,1.0,CC,27.0
15,2046923.0,PL,RMV,NaN,3525150,CHI,CHD,NaN,NaN,T,2004-08-09,30MAR2005,2005-04-13,1.0,GPK,30.0
16,2046923.0,PL,RMV,NaN,3538044,CHI,CHI,O,R,NaN,2004-08-09,13APR2005,2007-06-04,1.0,CC,27.0
17,2046924.0,MX,RMV,NaN,3200052,CHI,CHD,O,X,NaN,2004-08-09,13AUG2004,2004-08-13,NaN,RDV,31.0
18,2046925.0,MX,RMV,NaN,3200053,CHI,CHD,O,X,NaN,2004-08-10,19AUG2004,2004-08-19,NaN,JLG,29.0
19,2046926.0,MX,RMV,NaN,3200054,CHI,CHD,O,X,NaN,2004-08-10,16AUG2004,2004-08-30,NaN,CMZ,32.0
20,2046927.0,MX,RMV,NaN,3200055,CHI,CHD,O,X,NaN,2004-08-12,19AUG2004,2004-08-19,NaN,JLG,29.0


In [17]:
# delete duplicates (since idnproceeding are unique, this shouldn't do anything)
master = master.drop_duplicates(subset=['idncase', 'idnproceeding'])

In [18]:
master.count()

idncase             5669748
nat                 5667915
case_type           5669747
c_asy_type          1515478
idnproceeding       5669748
base_city_code      5669737
hearing_loc_code    5669646
dec_type            4534187
dec_code            4379061
other_comp          1290572
osc_date            5669748
input_date          5667795
comp_date           5669748
attorney_flag       2743993
ij_code             5637391
tracid              5360370
dtype: int64

In [19]:
master['idnproceeding'] = master['idnproceeding'].astype('float64')


In [20]:
#replace nan attorney flags with 0.
master.loc[pd.isnull(master.attorney_flag),'attorney_flag']=0


## Merge datasets

In [21]:
merged = pd.merge(app2, master, on=['idnproceeding','idncase'])

In [22]:
merged.count()

idnproceeding         608560
idnProceedingAppln    608560
idncase               608560
Appl_Code             608560
Appl_Recd_Date        608559
Appl_Dec              608560
numAppsPerProc        608560
dec                   608560
nat                   608403
case_type             608560
c_asy_type            595298
base_city_code        608558
hearing_loc_code      608557
dec_type              605465
dec_code              605170
other_comp              3353
osc_date              608560
input_date            608547
comp_date             608560
attorney_flag         608560
ij_code               607676
tracid                594055
dtype: int64

In [23]:
merged.head(10)

,idnproceeding,idnProceedingAppln,idncase,Appl_Code,Appl_Recd_Date,Appl_Dec,numAppsPerProc,dec,nat,case_type,...,hearing_loc_code,dec_type,dec_code,other_comp,osc_date,input_date,comp_date,attorney_flag,ij_code,tracid
0,75.0,41,3328085.0,ASYL,1994-12-20 00:00:00,D,3.0,DENY,HO,DEP,...,PIS,O,V,NaN,1994-11-03,10NOV1994,1995-03-10,1.0,CAL,NaN
1,85.0,43,3328111.0,ASYL,1995-04-02 00:00:00,D,3.0,DENY,HO,DEP,...,HOU,O,V,NaN,1994-11-04,14DEC1994,1997-06-16,1.0,WKZ,71.0
2,103.0,49,3328153.0,ASYL,1995-05-05 00:00:00,G,2.0,GRANT,GT,DEP,...,NYC,W,D,NaN,1994-11-05,20DEC1994,1995-08-08,1.0,JSC,139.0
3,111.0,52,3328175.0,ASYL,1995-03-31 00:00:00,D,3.0,DENY,ES,DEP,...,HOU,O,V,NaN,1994-11-05,06DEC1994,1995-08-15,1.0,CMR,70.0
4,136.0,56,3327844.0,ASYL,1995-03-06 00:00:00,D,3.0,DENY,HO,DEP,...,HLG,O,D,NaN,1995-02-06,06MAR1995,1995-04-06,1.0,JZ,50.0
5,139.0,58,3327852.0,ASYL,1996-05-14 00:00:00,G,2.0,GRANT,CU,DEP,...,MIA,O,R,NaN,1995-02-06,05OCT1995,1996-05-14,1.0,RAJ,126.0
6,145.0,60,3327869.0,ASYL,1995-11-29 00:00:00,D,3.0,DENY,NU,DEP,...,MIA,O,V,NaN,1995-02-10,23MAY1995,1996-01-04,1.0,WKZ,71.0
7,147.0,61,3327877.0,ASYL,1995-11-29 00:00:00,D,3.0,DENY,NU,DEP,...,MIA,O,V,NaN,1995-02-10,12APR1995,1996-01-04,1.0,WKZ,71.0
8,149.0,62,3327884.0,ASYL,1995-04-14 00:00:00,D,3.0,DENY,NU,DEP,...,PIS,O,V,NaN,1995-02-12,22FEB1995,1995-06-23,0.0,MB,61.0
9,159.0,63,3327927.0,ASYL,1995-07-25 00:00:00,D,3.0,DENY,NU,DEP,...,HOU,O,V,NaN,1995-02-17,15MAR1995,1995-09-05,1.0,CMR,70.0


In [24]:
#drop nan tracids and nat
merged = merged.dropna(subset=['tracid','nat'])
merged.count()


idnproceeding         593907
idnProceedingAppln    593907
idncase               593907
Appl_Code             593907
Appl_Recd_Date        593906
Appl_Dec              593907
numAppsPerProc        593907
dec                   593907
nat                   593907
case_type             593907
c_asy_type            580736
base_city_code        593907
hearing_loc_code      593906
dec_type              591161
dec_code              590868
other_comp              3002
osc_date              593907
input_date            593894
comp_date             593907
attorney_flag         593907
ij_code               593907
tracid                593907
dtype: int64

In [25]:
 merged.groupby('tracid').count()

,idnproceeding,idnProceedingAppln,idncase,Appl_Code,Appl_Recd_Date,Appl_Dec,numAppsPerProc,dec,nat,case_type,...,base_city_code,hearing_loc_code,dec_type,dec_code,other_comp,osc_date,input_date,comp_date,attorney_flag,ij_code
tracid,,,,,,,,,,,,,,,,,,,,,
1.0,4776,4776,4776,4776,4776,4776,4776,4776,4776,4776,...,4776,4776,4737,4737,38,4776,4776,4776,4776,4776
2.0,1244,1244,1244,1244,1244,1244,1244,1244,1244,1244,...,1244,1244,1237,1236,8,1244,1244,1244,1244,1244
3.0,4193,4193,4193,4193,4193,4193,4193,4193,4193,4193,...,4193,4193,4186,4182,11,4193,4192,4193,4193,4193
4.0,5731,5731,5731,5731,5731,5731,5731,5731,5731,5731,...,5731,5731,5708,5708,23,5731,5731,5731,5731,5731
5.0,878,878,878,878,878,878,878,878,878,878,...,878,878,877,871,7,878,878,878,878,878
6.0,624,624,624,624,624,624,624,624,624,624,...,624,624,623,622,2,624,624,624,624,624
7.0,5137,5137,5137,5137,5137,5137,5137,5137,5137,5137,...,5137,5137,5108,5111,26,5137,5135,5137,5137,5137
8.0,3503,3503,3503,3503,3503,3503,3503,3503,3503,3503,...,3503,3503,3493,3476,26,3503,3503,3503,3503,3503
9.0,2312,2312,2312,2312,2312,2312,2312,2312,2312,2312,...,2312,2312,2311,2308,4,2312,2312,2312,2312,2312


In [26]:
#drop all cases where judge has fewer than 100 cases--same as in gambler's fallacy paper
tracid_100 = merged.groupby('tracid').idnproceeding.count()>=100 #bool indicating whether judge has at least 100 cases
tracid_100 = tracid_100.index.values[tracid_100]#indices of judges with at least 100 cases
merged2 = merged.loc[merged.tracid.isin(tracid_100)]
merged2.count()

idnproceeding         592964
idnProceedingAppln    592964
idncase               592964
Appl_Code             592964
Appl_Recd_Date        592963
Appl_Dec              592964
numAppsPerProc        592964
dec                   592964
nat                   592964
case_type             592964
c_asy_type            580004
base_city_code        592964
hearing_loc_code      592963
dec_type              590219
dec_code              589927
other_comp              3000
osc_date              592964
input_date            592952
comp_date             592964
attorney_flag         592964
ij_code               592964
tracid                592964
dtype: int64

In [27]:
#there are 6377 osc dates before 1985, but I thought 1985 was supposed to be earliest year. 
merged2[merged2.osc_date.dt.year<1985]
#when osc year is pre 1985, it is often many years before the appl_recd_date. some may be error, but some may 
#actually have occurred well before the court date?


,idnproceeding,idnProceedingAppln,idncase,Appl_Code,Appl_Recd_Date,Appl_Dec,numAppsPerProc,dec,nat,case_type,...,hearing_loc_code,dec_type,dec_code,other_comp,osc_date,input_date,comp_date,attorney_flag,ij_code,tracid
12876,91492.0,27740,2048164.0,ASYL,1985-03-04 00:00:00,D,5.0,DENY,??,DEP,...,PHI,W,D,NaN,1984-03-13,11APR1984,1987-10-29,0.0,JFG,10.0
12908,93370.0,28338,2049203.0,ASYL,1988-03-31 00:00:00,D,3.0,DENY,ES,DEP,...,SFR,O,D,NaN,1984-03-21,31MAR1988,1988-12-01,1.0,BHS,202.0
12916,94058.0,28353,2049102.0,ASYL,1988-08-01 00:00:00,D,2.0,DENY,CU,EXC,...,OAK,W,E,NaN,1984-03-14,18MAY1988,1988-09-28,0.0,JAD,166.0
13052,99651.0,27838,2051255.0,ASYL,1990-03-23 00:00:00,D,3.0,DENY,PK,DEP,...,SFR,W,D,NaN,1984-07-17,23MAR1990,1990-09-28,1.0,DLM,197.0
13098,101817.0,27781,2052487.0,ASYL,1986-05-21 00:00:00,D,2.0,DENY,CU,DEP,...,MIA,W,D,NaN,1984-01-09,21MAY1986,1986-08-29,1.0,NSF,251.0
13186,106971.0,27974,2055405.0,ASYL,1990-05-15 00:00:00,D,2.0,DENY,CU,DEP,...,KRO,O,D,NaN,1984-05-14,01FEB1990,1991-01-10,0.0,NSF,251.0
13191,107019.0,27986,2055618.0,ASYL,1988-10-25 00:00:00,D,4.0,DENY,CU,DEP,...,MIA,W,D,NaN,1984-04-10,25OCT1988,1989-06-27,1.0,JMB,1.0
13192,107020.0,27987,2055618.0,ASYL,1985-02-28 00:00:00,D,4.0,DENY,CU,DEP,...,MIA,W,D,NaN,1984-04-10,25APR1984,1987-05-28,1.0,JMB,1.0
13265,110572.0,28123,2057635.0,ASYL,1988-12-13 00:00:00,D,4.0,DENY,KE,DEP,...,BOS,O,V,NaN,1984-09-04,18OCT1988,1989-05-03,1.0,BWD,226.0
13266,110573.0,28124,2057635.0,ASYL,1986-04-10 00:00:00,D,4.0,DENY,KE,DEP,...,BOS,O,D,NaN,1984-09-04,17OCT1985,1986-10-02,1.0,BWD,226.0


In [28]:
#look at comp date--626 dates before 1985. drop these.
len(merged2[merged2.comp_date.dt.year<1985])
merged2 = merged2[merged2.comp_date.dt.year>1984]
merged2.count()

idnproceeding         592964
idnProceedingAppln    592964
idncase               592964
Appl_Code             592964
Appl_Recd_Date        592963
Appl_Dec              592964
numAppsPerProc        592964
dec                   592964
nat                   592964
case_type             592964
c_asy_type            580004
base_city_code        592964
hearing_loc_code      592963
dec_type              590219
dec_code              589927
other_comp              3000
osc_date              592964
input_date            592952
comp_date             592964
attorney_flag         592964
ij_code               592964
tracid                592964
dtype: int64

In [29]:
# adding additional feature based on how many asylum proceedings have been filed for the same (idnCase) 
merged2['numProcPerCase'] = 1
merged2['numProcPerCase'] = merged2['numProcPerCase'].astype('int64')
merged2['numProcPerCase'] = merged2.groupby(['idncase'])['numProcPerCase'].transform('count')

#make unique at idncase level, sorting with the same logic as used to sort applications
#counting case as a grant if ANY proceeding was grant
merged_case = merged2.sort_values(['idncase','dec','Appl_Code','Appl_Recd_Date'],ascending=[True,False,True,False])
merged_case = merged_case.groupby('idncase',as_index=False ).first()

In [30]:
merged_case.groupby('dec').count()

,idncase,idnproceeding,idnProceedingAppln,Appl_Code,Appl_Recd_Date,Appl_Dec,numAppsPerProc,nat,case_type,c_asy_type,...,dec_type,dec_code,other_comp,osc_date,input_date,comp_date,attorney_flag,ij_code,tracid,numProcPerCase
dec,,,,,,,,,,,,,,,,,,,,,
DENY,338928,338928,338928,338928,338928,338928,338928,338928,338928,328535,...,337399,337399,1687,338928,338918,338928,338928,338928,338928,338928
GRANT,226084,226084,226084,226084,226083,226084,226084,226084,226084,223782,...,225242,225027,1309,226084,226082,226084,226084,226084,226084,226084


In [31]:
unique_nat = merged_case.nat.unique()
unique_nat

array(['CH', 'CM', 'AL', 'ET', 'CF', 'LE', 'GT', 'HO', 'IN', 'UE', 'CU',
       'LV', 'ES', 'UZ', 'LH', 'MX', 'AU', 'PO', '??', 'JM', 'RU', 'CO',
       'UK', 'RO', 'HU', 'IR', 'TS', 'ID', 'SY', 'YO', 'EC', 'PK', 'CZ',
       'NU', 'DR', 'JA', 'PE', 'BU', 'VE', 'GE', 'MO', 'LI', 'HK', 'TW',
       'HA', 'CS', 'NI', 'AR', 'RP', 'IZ', 'PL', 'YS', 'KE', 'BH', 'JO',
       'AG', 'ST', 'PM', 'EG', 'GR', 'BN', 'TU', 'AZ', 'FJ', 'BG', 'BY',
       'IV', 'BI', 'CI', 'CE', 'GH', 'KS', 'CX', 'SL', 'BM', 'GY', 'MV',
       'UR', 'GO', 'GV', 'CY', 'IS', 'AF', 'NN', 'BL', 'EI', 'CG', 'BW',
       'TD', 'CA', 'CQ', 'SO', 'VM', 'KU', 'MM', 'IT', 'SF', 'UG', 'AM',
       'YE', 'UY', 'BX', 'GA', 'LY', 'SK', 'BF', 'BR', 'PA', 'FG', 'CB',
       'SS', 'PS', 'CC', 'LA', 'BB', 'ZA', 'SZ', 'IY', 'AO', 'PU', 'TH',
       'SW', 'ZI', 'KV', 'SN', 'WS', 'MT', 'BA', 'MZ', 'SU', 'GJ', 'KN',
       'FR', 'MY', 'TZ', 'NG', 'SM', 'ER', 'TO', 'SA', 'SP', 'MA', 'CV',
       'MI', 'BC', 'RW', 'NS', 'GZ', 'SG', 'CT', 'N

In [32]:
#drop 159 cases with unknown nationalities
merged_case = merged_case.loc[~(merged_case.nat=='??')]


In [33]:
#get rid of merged_cases where other_comp is not null. other_comp indicates that the proceeding ended for a reason other than 
#a judge's decision, suggesting no decision was actually made. this is less than 1% of cases once we have already filtered
#out applications where the decision is not grant or deny and matched them to proceedings.
merged_case = merged_case[pd.isnull(merged_case.other_comp)]

#get rid of cases that don't have c_asy type (about 2% of cases--higher proportion than full_asylum version...)
merged_case = merged_case[~pd.isnull(merged_case.c_asy_type)] 

#change values of c_asy_type to be more clear
merged_case.loc[merged_case.c_asy_type=='I','c_asy_type'] = 'aff'
merged_case.loc[merged_case.c_asy_type=='E','c_asy_type'] = 'def'

In [34]:
merged_case.count()

idncase               549320
idnproceeding         549320
idnProceedingAppln    549320
Appl_Code             549320
Appl_Recd_Date        549319
Appl_Dec              549320
numAppsPerProc        549320
dec                   549320
nat                   549320
case_type             549320
c_asy_type            549320
base_city_code        549320
hearing_loc_code      549319
dec_type              548986
dec_code              549278
other_comp                 0
osc_date              549320
input_date            549308
comp_date             549320
attorney_flag         549320
ij_code               549320
tracid                549320
numProcPerCase        549320
dtype: int64

In [35]:
#drop variables that definitely won't be used as features (or won't be used to track where the data came from)
merged_case = merged_case.drop(['Appl_Dec','Appl_Code','Appl_Recd_Date','dec_type','other_comp','input_date','ij_code','dec_code'],axis=1)


In [36]:
#load nationality lookup table
nat_lut =  pd.read_csv(path+ '/tblLookupNationality.csv',header=None)

#drop 4 observations where the nationality code is not in the lookup table
merged_case = merged_case.loc[merged_case.nat.isin(nat_lut[1])]

#drop 2 observations with nationality code XX whic the LUT says corresponds 
#to "BE REMOVED FROM THE UNITED STATES"
merged_case = merged_case.loc[~(merged_case.nat=="XX")]

#examine counts for different nationalities:
nat_numbers = merged_case.groupby('nat',as_index=False).idncase.count().sort_values('idncase')

# some nationalities have only 1 or 2 observations. drop any with less than 10 observations.
nat_10 = nat_numbers.loc[nat_numbers['idncase']>9,'nat']
merged_case = merged_case[merged_case.nat.isin(nat_10)]

In [37]:
#remove cities with less than 10 obs (only removes 2 cases)

city_numbers = merged_case.groupby('base_city_code',as_index=False).idncase.count().sort_values('idncase')
cities_10 = city_numbers.loc[city_numbers['idncase']>9,'base_city_code']

merged_case = merged_case[merged_case.base_city_code.isin(cities_10)]


In [38]:
#hearing loc--is this "court"? some of them are prisons/detention centers/airports. many "courts" have fewer than 10 obs
#drop thewse (less than 1% of proceedings)

court_numbers = merged_case.groupby('hearing_loc_code',as_index=False).idncase.count().sort_values('idncase')
courts_10 = court_numbers.loc[court_numbers['idncase']>9,'hearing_loc_code']
merged_case = merged_case[merged_case.hearing_loc_code.isin(courts_10)]


In [39]:
merged_case.groupby('dec').count()

,idncase,idnproceeding,idnProceedingAppln,numAppsPerProc,nat,case_type,c_asy_type,base_city_code,hearing_loc_code,osc_date,comp_date,attorney_flag,tracid,numProcPerCase
dec,,,,,,,,,,,,,,
DENY,326581,326581,326581,326581,326581,326581,326581,326581,326581,326581,326581,326581,326581,326581
GRANT,222361,222361,222361,222361,222361,222361,222361,222361,222361,222361,222361,222361,222361,222361


In [40]:
#save data
merged_case.to_csv('/home/emilyboeke/merged_any_master_app.csv',index=False)